# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = 'llama3.2'
openai = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

API key looks good so far


In [5]:
links = fetch_website_links("http://cargill.com")
links

['#maincontent',
 'https://www.cargill.com/home',
 'https://www.cargill.com/home',
 'https://www.cargill.com/about',
 'https://www.cargill.com/about',
 'https://www.cargill.com/about/our-approach',
 'https://www.cargill.com/about/cargill-at-a-glance',
 'https://www.cargill.com/about/executive-team',
 'https://www.cargill.com/about/innovation',
 'https://www.cargill.com/about/2025-annual-report',
 'https://www.cargill.com/about/cargill-history-timeline',
 'https://www.cargill.com/about/community-engagement',
 'https://www.cargill.com/about/research/research-development',
 'https://www.cargill.com/about/diversity-equity-inclusion',
 'https://www.cargill.com/about/workplace-safety',
 'https://www.cargill.com/about/ethics-and-compliance',
 'https://www.cargill.com/about/strategic-sourcing',
 'https://www.cargill.com/page/cargill-contact-us',
 'https://www.cargill.com/news/cargill-stories',
 'https://www.cargill.com/products',
 'https://www.cargill.com/products',
 'https://www.cargill.com/a

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://cargill.com"))


Here is the list of links on the website https://cargill.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

#maincontent
https://www.cargill.com/home
https://www.cargill.com/home
https://www.cargill.com/about
https://www.cargill.com/about
https://www.cargill.com/about/our-approach
https://www.cargill.com/about/cargill-at-a-glance
https://www.cargill.com/about/executive-team
https://www.cargill.com/about/innovation
https://www.cargill.com/about/2025-annual-report
https://www.cargill.com/about/cargill-history-timeline
https://www.cargill.com/about/community-engagement
https://www.cargill.com/about/research/research-development
https://www.cargill.com/about/diversity-equity-inclusion
https://www.cargill.com/about/workplace-safety
https://www.cargill.com/about/ethics-and-compliance
https://www.cargill.com/a

In [10]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://cargill.com")

{'links': [{'type': 'home page', 'url': 'https://www.cargill.com/home'},
  {'type': 'about page', 'url': 'https://www.cargill.com/about'},
  {'type': 'careers page', 'url': 'https://careers.cargill.com/'},
  {'type': 'news page', 'url': 'https://www.cargill.com/news'},
  {'type': 'products page', 'url': 'https://www.cargill.com/products'},
  {'type': 'community engagement page',
   'url': 'https://www.cargill.com/about/community-engagement'},
  {'type': 'sustainability page (2025 Impact Report)',
   'url': 'https://www.cargill.com/sustainability/2025-impact-report'},
  {'type': 'sustainability reporting hub',
   'url': 'https://www.cargill.com/sustainability/reporting-hub'},
  {'type': 'news (exclusive interview) page',
   'url': 'https://www.cargill.com/news/exclusive-interview-jarrod-gillig'},
  {'type': 'worldwide office',
   'url': 'https://www.cargill.com/page/worldwide'}]}

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [12]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Lightricks/LTX-2
Updated
2 days ago
•
497k
•
726
tencent/HY-MT1.5-1.8B
Updated
9 days ago
•
9.06k
•
698
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
3 days ago
•
10.3k
•
310
nvidia/nemotron-speech-streaming-en-0.6b
Updated
5 days ago
•
1.86k
•
263
naver-hyperclovax/HyperCLOVAX-SEED-Think-32B
Updated
4 days ago
•
30.3k
•
335
Browse 2M+ models
Spaces
Running
Featured
4.08k
Wan2.2 Animate
👁
4.08k
Wan2.2 Animate
Running
on
Zero
305
Qwen Image Multiple Angles 3D Camera
🎥
305
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
1.24k
Z Image Turbo
🖼
1.24k
Generate stunning AI images from text description

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

ConnectionError: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x10ebcaf00>: Failed to resolve 'blog.huggingface.co' ([Errno 8] nodename nor servname provided, or not known)"))

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("Edward Donner", "https://edwarddonner.com")

**Edward Donner: Revolutionizing the Future of Work**

At Edward Donner, we're passionate about harnessing the power of AI to unlock human potential and create a more fulfilling world. Our mission is to help people discover their purpose and find roles that align with their values, leading to greater prosperity and happiness.

**Our Story**

Our founder and CTO, Ed, has been at the forefront of AI innovation for over two decades. With a background in building successful Tech startups, he's seen firsthand the challenges faced by recruiters and job seekers. Ed's vision is to use Generative AI and other machine learning technologies to create a more personalized and effective hiring process.

**Our Technology**

Our patented model uses AI to match people with roles in real-time, eliminating the need for tedious keyword searches. This technology has been deployed to help recruiters source top talent, and we're committed to sharing its potential with the world.

**Our Values**

At Edward Donner, we value collaboration, creativity, and a passion for making a positive impact. We believe that AI can be used to benefit humanity, rather than control it. Our culture is built around:

* **Innovation**: We're constantly pushing the boundaries of what's possible with AI
* **Expertise**: Our team is comprised of experienced professionals with deep knowledge in various fields
* **Community**: We're committed to building strong relationships with our partners and stakeholders

**Careers**

Join us on this exciting journey! We offer a range of opportunities for talented individuals who share our vision. Check out our current job openings and learn more about the careers at Edward Donner.

### Link to Career Page (Not available)

Unfortunately, I do not have access to the career page for Edward Donner as it is not provided. 

### Additional Information

You can follow us on LinkedIn, Twitter, or Facebook to stay up-to-date with our latest news and updates. Subscribe to our newsletter to get insights into the world of AI and job market trends.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("Edward Donner", "https://edwarddonner.com")

# Edward Donner Brochure

## About Us

Edward Donner is a pioneering AI company founded by Ed Donner, a renowned expert in Artificial Intelligence. Our mission is to empower people to discover their potential and pursue their reason for being through the application of AI in real-world problems.

We are dedicated to helping recruiters source, understand, engage, and manage talent using Generative AI and other forms of machine learning. Our patented model matches people with roles with greater accuracy and speed than previously imaginable – no keywords required.

## Company Culture

At Edward Donner, we value innovation, collaboration, and a passion for making a positive impact. As an organization, we're committed to fostering a culture that encourages creativity, autonomy, and growth.

We believe in working on real-world problems and finding innovative solutions. Our long-term goal is to help people discover their potential and pursue their reason for being, motivated by the concept of Ikigai.

## Customers

We work with forward-thinking organizations that recognize the power of AI in solving real-world problems. By partnering with us, you'll gain access to our cutting-edge technology and expertise in Generative AI and talent management.

Our customers are at the forefront of industries such as recruitment, education, and research. With Edward Donner, you'll be able to:

* Source high-quality talent with unprecedented accuracy
* Understand your team's strengths and weaknesses
* Engage with your employees on a deeper level
* Make data-driven decisions that drive business value

## Careers/Jobs

Join our dynamic team of innovators who are passionate about applying AI to real-world problems. We're currently hiring for various roles, including:

* Talent Management Specialist: Work alongside Ed Donner to develop and deploy proprietary talent management solutions.
* AI Engineer: Contribute to the development of our Cutting Edge AI Solutions
* Recruitment Expert: Help us optimize our recruitment pipeline with innovative strategies.

Apply now to be part of a talented team that's shaping the future of work.

### Connect With Us

Stay connected with Edward Donner to learn more about our AI-powered solutions and stay updated on our latest news.

Follow us:
Social Media Platforms
Newsletter Sign-Up

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>